In [40]:
specfile=open('spec.txt','r',encoding='utf-8')
spec=specfile.read().split('\n\n')

def to_(origstr):
    string=origstr
    for i in range(len(string)):
        if 0x40<ord(string[i]) and ord(string[i])<0x61 and string[i]!='_':
            string=string[:i]+'_'+string[i].lower()+string[i+1:]
    return string
def writefile(infilename,replaceContent,outfilename):
    infile=open(infilename,'r',encoding='utf-8')
    string=infile.read()
    for content in replaceContent:
        string=string.replace(content[0],content[1])
    
    outfile=open(outfilename,'w',encoding='utf-8')
    outfile.write(string)
    outfile.close()
    infile.close()
typetrans1={
    "VARCHAR(25)":"string",
    "VARCHAR(255)":"string",
    "VARCHAR(45)":"string",
    "INT":"number",
    "VARCHAR(10)":"string",
    "varchar":"string",
    "DECIMAL ":"string",
    "varchar(255)":"string",
    "MONEY":"number",
    "VARCHAR(100)":"string",
    "FLOAT":"number",
    "varchar(50)":"string",
    "NUMERIC":"number",
    "INT ":"number",
    "numeric":"number",
    "DATE":"string",
    "DECIMAL(12, 2)":"string",
    "VARCHAR(3)":"string",
    "INT1(1 byte data type)":"number",
    "VARCHAR(60)":"string",
    "INT1(1byte data type)":"number",
    "NUMERIC(15)":"number",
    "VARCHAR(50)":"string",
    "BOOLEAN":"boolean",
    "DATETIME":"string",
    "INTEGER":"number"
}
typetrans2={
    "VARCHAR(25)":"DataTypes.STRING(25)",
    "VARCHAR(255)":"DataTypes.STRING(255)",
    "VARCHAR(45)":"DataTypes.STRING(45)",
    "VARCHAR(10)":"DataTypes.STRING(10)",
    "varchar":"DataTypes.STRING(255)",
    "varchar(255)":"DataTypes.STRING(255)",
    "VARCHAR(60)":"DataTypes.STRING(60)",
    "VARCHAR(100)":"DataTypes.STRING(100)",
    "varchar(50)":"DataTypes.STRING(50)",
    "VARCHAR(3)":"DataTypes.STRING(3)",
    "VARCHAR(50)":"DataTypes.STRING(50)",
    
    "MONEY":"Types.MONEY",
    
    "DECIMAL ":"DataTypes.DECIMAL",
    "DECIMAL(12, 2)":"DataTypes.DECIMAL(12,2)",
    
    "FLOAT":"DataTypes.REAL",
    "NUMERIC":"DataTypes.REAL",
    "NUMERIC(15)":"DataTypes.REAL",
    "numeric":"DataTypes.REAL",
    
    "INT ":"DataTypes.INTEGER",
    "INTEGER":"DataTypes.INTEGER",
    "INT":"DataTypes.INTEGER",
    "INT1(1 byte data type)":"DataTypes.INTEGER",
    "INT1(1byte data type)":"DataTypes.INTEGER",
    
    "BOOLEAN":"DataTypes.BOOLEAN",

    "DATETIME":"DataTypes.DATE",
    "DATE":"DataTypes.DATEONLY"
}
db=""
indeximportfile=""

useroute=""
routeimportfile=""
for item in spec:
    isJournal="_isjournal" if item.find("isJournal")>0 else""
    
    item=item.split('\n')
    specname=item[0].split()
    name=specname[0]
    smallname=specname[2]
    bigname=name
    bigname=name[0].capitalize()+bigname[1:]
    name_name=to_(name)
    zhname=specname[1]
    
    print("%s\t%s\t%s"%(name,zhname,smallname))
    
    item.pop(0)
    swagger_part=""
    bodylist=""
    reqlist=""
    
    attributes=""
    count=0
    for col in item:
        count+=1
        col=col.split('\t')

        try:
            datatype_json=typetrans1[col[2]]
            datatype_db=typetrans2[col[2]]
            if col[1].find("Code")>0:
                datatype_db="Types.CODE"
            elif col[1].find("Address")>0:
                datatype_db="Types.ADDRESS"
            elif col[1].find("Email")>0:
                datatype_db="Types.EMAIL"
        except:
            datatype_json="string"
            datatype_db="DataTypes.STRING(255)"
            if col[1].find("Code")>0:
                datatype_db="Types.CODE"
            elif col[1].find("Address")>0:
                datatype_db="Types.ADDRESS"
            elif col[1].find("Email")>0:
                datatype_db="Types.EMAIL"
                
        if datatype_json=="number":
            default="0"
        elif datatype_json=="boolean":
            default="true"
        else:
            default=col[0]
        islast="" if count==len(item) else ","
        swagger_part+='\t\t\t\t\t\t\t%s:{ type: "%s", default: "%s" }%s\n'%(col[1],datatype_json,default,islast)
        reqlist+="\t\t%s:req.body.%s,\n"%(col[1],col[1])
        bodylist+='"%s"%s'%(col[1],islast)
    

        attributes+="""\t\t\t%s: {
    \t\t\ttype: %s
    \t\t},\n"""%(col[1],datatype_db)

    replacController=[
        ["_bigname_",bigname],
        ["_name_",name],
        ["zhname",zhname],
        ["_bodylist_",bodylist],
        ["swagger_part",swagger_part],
        ["_reqlist_",reqlist]
    ]
    writefile("editTemplate%s.js"%isJournal,replacController,"controllers\edit%s.controller.js"%bigname)

    replaceModel=[
        ['_smallname_',smallname],
        ['_bigname_',bigname],
        ['_name_',name],
        ['_attributes_',attributes]
    ]
    writefile("modelTemplate.js",replaceModel,"models/%s.js"%name_name)

    replaceTest=[
        ['_smallname_',smallname],
        ["_zhname_",zhname],
        ['_name_',name],
        ['_reqlist_',bodylist]
    ]
    writefile("testTemplate.js",replaceTest,"test/edit%s.test.js"%bigname)

    replaceRoute=[
        ['_smallname_',smallname],
        ['_bigname_',bigname],
        ['_name_',name]
    ]
    writefile('routeTemplate%s.js'%isJournal,replaceRoute,"route/edit%s.js"%bigname)

    indeximportfile+='import %s from "./%s.js";\n'%(name_name,name_name)
    db+='\t%s: %s(sequelize),\n'%(name,name_name)

    routeimportfile+="import Edit%sRouter from './edit%s.js';"%(bigname,bigname)
    useroute+="EditRouter.use('/%s', Edit%sRouter);"%(name,bigname)
    
replaceIndex=[
    ['_import_',indeximportfile],
    ['_db_',db]
]
writefile('indexTemplate.js',replaceIndex,'models/index.js')

replaceEditRoute=[
    ['_import_',routeimportfile],
    ['_useroute_',useroute]
]
writefile('editRouteTemplate.js',replaceEditRoute,'route/edit.js')

todo	代辦事項	todo
financialStatement	簡明三表	fs
balanceSheet	資產負債表	bs
incomeStatement	損益表	is
cashflowStatement	現金流量表	cfs
faTransaction	固定資產買賣表	fa
fixedAssets	固定資產變動表	fa
account	會計項目	account
journal	分錄	journal
alarm	預警	alarm
user	使用者	user
department	部門	dept
employee	員工	emp
customer	買受人(客戶)	cust
bank	銀行帳戶	bank
loan	銀行借貸表	loan
cashflow	現金帳	cashflow
company	公司	company
product	產品	prod
inv	庫存盤點	inv
vendor	供應廠商(含合作商)	vendor
paymentMethod	付款方法	pm
pettyCash	零用金代墊	pc
poReceived	採購收料	poRcv
soReceivable	產品銷貨紀錄	soRcv
returnsAndAllowances	折讓退貨表	sra
project	專案	proj
salaries	薪資列表	salaries
shop	店家	shop
storage	倉儲	storage
business_unit	事業體	bizUnit
expense	成本列表	exp
salesTarget	銷售預估	salesTarget
poForecast	進貨預估	poFcst
insurance	保險	insurance
upload	上傳	upload


In [3]:
specfile=open('spec.txt','r',encoding='utf-8')
spec=specfile.read().split('\n\n')
for item in spec:
    item=item.split('\n')
    term=item[0].split()[0]
    print("%s: %s(sequelize),"%(term,term))

todo: todo(sequelize),
financialStatement: financialStatement(sequelize),
balanceSheet: balanceSheet(sequelize),
incomeStatement: incomeStatement(sequelize),
cashflowStatement: cashflowStatement(sequelize),
faTransaction: faTransaction(sequelize),
fixedAssets: fixedAssets(sequelize),
account: account(sequelize),
journal: journal(sequelize),
alarm: alarm(sequelize),
user: user(sequelize),
dept: dept(sequelize),
emp: emp(sequelize),
cust: cust(sequelize),
bank: bank(sequelize),
loan: loan(sequelize),
cashflow: cashflow(sequelize),
company: company(sequelize),
prod: prod(sequelize),
inv: inv(sequelize),
vendor: vendor(sequelize),
paymentMethod: paymentMethod(sequelize),
pc: pc(sequelize),
poRcv: poRcv(sequelize),
soRcv: soRcv(sequelize),
returnsAndAllowances: returnsAndAllowances(sequelize),
proj: proj(sequelize),
salaries: salaries(sequelize),
shop: shop(sequelize),
storage: storage(sequelize),
business_unit: business_unit(sequelize),
expense: expense(sequelize),
salesTarget: salesTarget